In [1]:
import pickle
import torch
from transformers import AutoModel, AutoTokenizer
from tqdm import tqdm 

In [22]:
with open("news_dict.pickle","rb") as f :
    data = pickle.load(f)

In [13]:
len_list = []
for key, val in data.items():
    len_list.append((key, len(val)))

In [16]:
data2 = sorted(len_list, key = lambda x: x[1],reverse=True)

In [21]:
data2[:10]

[('대상', 65640),
 ('레이', 23658),
 ('SK', 7550),
 ('동양', 7110),
 ('LG', 6228),
 ('신한', 6142),
 ('디오', 6037),
 ('성원', 5473),
 ('동아', 5022),
 ('남성', 4812)]

In [2]:
from konlpy.tag import Mecab
tokenizer = Mecab()

In [23]:
nouns = {}
for key, vals in tqdm(data.items()):
    tot = []
    for txt in vals:
        txt_list = tokenizer.nouns(txt)
        if key in txt_list :
            tot.append(txt)
    nouns[key] = tot

100%|██████████| 2592/2592 [12:19<00:00,  3.50it/s] 


In [25]:
with open("news_dict_nouns.pickle","wb") as f:
    pickle.dump(nouns, f)

In [26]:
len_list = []
for key, val in nouns.items():
    len_list.append((key, len(val)))

In [49]:
del(nouns['전방'])
del(pre_dict['전방'])

In [43]:
del(pre_dict['대상'])

In [27]:
data = sorted(len_list, key = lambda x: x[1],reverse=True)

In [34]:
pre_dict = {}
for key, vals in nouns.items():
    pre_list = []
    if len(vals) == 0:
        continue
    for val in vals:
        pre_list.append(val.replace(key,"[MASK]"))
    pre_dict[key] = pre_list

In [37]:
import pandas as pd

In [52]:
col_list = ["corp","plain","masked"]

In [53]:
data_list = []
for key in nouns.keys():
    for idx in range(len(nouns[key])):
        data_list.append([key, nouns[key][idx],pre_dict[key][idx]])

In [54]:
len(data_list)

74396

In [55]:
df = pd.DataFrame(data_list, columns=col_list)

In [57]:
with open("dataset.pickle","wb") as f:
    pickle.dump(df, f)

In [58]:
from sklearn.model_selection import train_test_split

In [194]:
train, test, _, _= train_test_split(
    df,df['corp'], test_size=0.3, random_state=42)

In [195]:
valid, test, _, _= train_test_split(
    test,test['corp'], test_size=0.5, random_state=42)

In [64]:
label_to_corp = {idx:list(nouns.keys())[idx] for idx in range(len(list(nouns.keys())))}
corp_to_label = {val:idx for idx, val in label_to_corp.items()}

In [73]:
news_length = [len(txt) for txt in df['plain'].values]
max(news_length)

13678

In [161]:
import datasets
dartdataset = datasets.load_dataset('nlpHakdang/beneficiary',  data_files = "dart_v3_01.csv", use_auth_token='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')


Using custom data configuration nlpHakdang___beneficiary-f7051ac6fd8f9af6


  0%|          | 0/1 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /opt/ml/.cache/huggingface/datasets/csv/nlpHakdang___beneficiary-f7051ac6fd8f9af6/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [139]:
def preprocessing_only_kr(s): 
        hangul = re.compile('[^ ㄱ-ㅣ가-힣+]')
        result = hangul.sub('', s)
        return ' '.join(result.split())

def preprocessing(s): 
        hangul = re.compile('[^ a-zA-Z0-9ㄱ-ㅣ가-힣+]')
        result = hangul.sub('', s)
        return ' '.join(result.split())

In [171]:
preprocess = [preprocessing(txt) for txt in dartdataset['train']['사업의 개요']]
dartdata = [dartdataset['train']['기업 이름'],preprocess]
dartdf = pd.DataFrame (dartdata).transpose()
dartdf.columns = ['corp', 'txt']

### Stride Code
#### DART
- 데이터의 형태 colum = ['corp', 'txt']
    - corp : 기업이름, txt : 사업의 개요
- corp_to_label : 회사이름을 정수로 바꾸는 딕셔너리

In [233]:
def dart_split(df : DataFrame) -> pd.DataFrame : 
    data_list2 = []
    for idx in tqdm(range(len(df))):
        stride = []
        txt = df.iloc[idx]['txt']
        if len(txt) < 512:
            if df.iloc[idx]['corp'] not in corp_to_label:
                continue
            data_list2.append([corp_to_label[df.iloc[idx]['corp']],df.iloc[idx]['txt']])
        else:
            for pointer in range(len(txt)//300):
                if pointer == len(txt)//300-1:
                    split_txt = txt[-400:]
                if pointer != 0:
                    split_txt = txt[(pointer*300)-100:((pointer+1)*300)+100]
                if pointer == 0:
                    split_txt = txt[:400]
                if df.iloc[idx]['corp'] not in corp_to_label:
                    continue
                data_list2.append([corp_to_label[df.iloc[idx]['corp']],df.iloc[idx]['txt']])
    return pd.DataFrame(data_list2,columns=['corp', 'txt'])

#### News
- 데이터의 형태 colum = ['corp', 'plain', 'maked']
    - corp : 기업이름, plain : 원본텍스트, masked:마스크된 텍스트
- corp_to_label : 회사이름을 정수로 바꾸는 딕셔너리

In [234]:
def bert_split(df : pd.DataFrame) -> pd.DataFrame:
    data_list2 = []
    for idx in tqdm(range(len(df))):
        stride = []
        txt = df.iloc[idx]['masked']
        if len(txt) < 512:
            data_list2.append([corp_to_label[df.iloc[idx]['corp']],df.iloc[idx]['plain'],df.iloc[idx]['masked']])
        else:
            for pointer in range(len(txt)//300):
                if pointer == len(txt)//300-1:
                    split_txt = txt[-400:]
                if pointer != 0:
                    split_txt = txt[(pointer*300)-100:((pointer+1)*300)+100]
                if pointer == 0:
                    split_txt = txt[:400]
                data_list2.append([corp_to_label[df.iloc[idx]['corp']],df.iloc[idx]['plain'],split_txt])
    return pd.DataFrame(data_list2,columns=['corp','plain','masked'])

In [207]:
train_df = bert_split(train)
valid_df = bert_split(valid)
test_df = bert_split(test)

100%|██████████| 11160/11160 [00:05<00:00, 1966.93it/s]


In [208]:
split_dart_df = dart_split(dartdf)

100%|██████████| 2410/2410 [00:04<00:00, 519.67it/s]


In [209]:
len(train), len(test), len(train_df), len(test_df), len(dartdf), len(split_dart_df)

(52077, 11160, 152704, 32702, 2410, 20643)

In [225]:
test_df.iloc[0:3]['masked'][0]

'서울시가 과열된 수주전으로 몸살을 앓고 있는 서울 용산구 한남3구역 재개발 정비사업의 시공사 입찰선정에 대한 특별점검을 진행한다3일 업계에 따르면 서울시는 국토교통부 한국감정원과 합동점검반을 꾸려 4일부터 15일까지 회계처리 정보공개 등 일반적 사항은 물론 최근 과열 기미를 보이는 수주경쟁에 대한 법 위반사항을 점검한다시공사가 입찰제안서에 명시한 설계 등 시공과 관련된 내용이 모두 점검대상에 포함됐다합동점검반은 서울시와 국토부 자치구의 정비사업 담당 공무원과 감정평가사 변호사 회계사 건설 분야별 전문가 등 14명이 참여할 예정이다점검과정에서 불법행위가 적발될 때에는 행정처분 등의 조치를 진행한다지난달 18일 진행한 입찰에는 현대건설 대림산업 GS건설 등 3개사가 참여했다현대건설은 현대백화점 그룹과 손잡고 단'

In [117]:
test_df.iloc[0:3]['masked'][1]

'해 진행되는 이번 프로모션은 세상에서 가장 편안한 SUV를 목표로 개발된 시트로엥 뉴 C5 에어크로스 SUV를 한 층 더 완벽하게 경험할 기회를 제공하기 위해 기획됐다시트로엥의 플래그십 SUV 뉴 C5 에어크로스 SUV는 오는 29일부터 4월 7일까지 킨텍스에서 개최되는 2019 서울모터쇼에서 국내 최초로 공개된다모터쇼 기간 동안 뉴 C5 에어크로스 SUV를 계약하는 고객 중 3명에겐 추첨을 통해 레이노 팬텀 S9 풀 시공권이 증정된다세계 최초의 [MASK] 카본 세라믹 기술로 탄생한 레이노 팬텀 틴팅 필름은 카본과 세라믹의 장점만을 융합하여 선명한 시인성과 우수한 변색 방지 내구성으로 사랑받고 있다특히 레이노 팬텀 S9은 적외선을 90% 이상 차단하여 뛰어난 열 차단 성능을 제공하는 프리미엄 제품으로 시트로엥 뉴 C5 에어크로스 SUV와 함께 편안하고 쾌적한 드라이빙 환경을 선사할 것으로 기대된다이 외에도 2019 서울모터쇼 기간 동안 시트로엥 부스에서 진행하는 다양한 이벤트에 참'

In [118]:
test_df.iloc[0:3]['masked'][2]

'을 융합하여 선명한 시인성과 우수한 변색 방지 내구성으로 사랑받고 있다특히 레이노 팬텀 S9은 적외선을 90% 이상 차단하여 뛰어난 열 차단 성능을 제공하는 프리미엄 제품으로 시트로엥 뉴 C5 에어크로스 SUV와 함께 편안하고 쾌적한 드라이빙 환경을 선사할 것으로 기대된다이 외에도 2019 서울모터쇼 기간 동안 시트로엥 부스에서 진행하는 다양한 이벤트에 참여하면 차량의 도장면을 보호하고 뛰어난 광택 성능을 제공하는 레이노 블라스크 세라믹 코트 또한 선물 받을 수 있다레이노 코리아의 신현일 지사장은 윈도우 필름 업계에서 새로운 기준을 제시하는 레이노 팬텀과 세상에서 가장 편안한 SUV를 표방하는 뉴 C5 에어크로스 SUV가 만나 소비자의 니즈를 충족하는 시너지를 창출했다며 앞으로도 시트로엥과의 지속적인 콜라보를 통해 고객들에게 만족을 줄 수 있도록 노력해 나가겠다고 밝혔다손균근 기자 kr'

In [226]:
test_df

,corp,plain,masked
0,922,서울시가 과열된 수주전으로 몸살을 앓고 있는 서울 용산구 한남3구역 재개발 정비사업...,서울시가 과열된 수주전으로 몸살을 앓고 있는 서울 용산구 한남3구역 재개발 정비사업...
1,922,서울시가 과열된 수주전으로 몸살을 앓고 있는 서울 용산구 한남3구역 재개발 정비사업...,시공과 관련된 내용이 모두 점검대상에 포함됐다합동점검반은 서울시와 국토부 자치구의 ...
2,343,데일리동방 우리은행이 모회사인 우리금융지주의 주식 상당수를 해외 투자자에게 순조롭게...,데일리[MASK] 우리은행이 모회사인 우리금융지주의 주식 상당수를 해외 투자자에게 ...
3,343,데일리동방 우리은행이 모회사인 우리금융지주의 주식 상당수를 해외 투자자에게 순조롭게...,진다앞서 우리금융은 우리카드를 지난 10일 우리금융 자회사로 편입했다우리금융이 우리...
4,229,경북지역 5개 대학 150명 대상 6월까지 두 달간경북지역 5개 대학 150명 대상...,경북지역 5개 대학 150명 대상 6월까지 두 달간경북지역 5개 대학 150명 대상...
...,...,...,...
32697,1183,건국대 글로컬캠퍼스 뉴미디어아트연구소와 이머시브미디어연구소 은이 주관하고 피터앤더울...,건국대 글로컬캠퍼스 뉴미디어아트연구소와 이머시브미디어연구소 은이 주관하고 피터앤더울...
32698,1183,건국대 글로컬캠퍼스 뉴미디어아트연구소와 이머시브미디어연구소 은이 주관하고 피터앤더울...,아티스트와 최신 기술분야 전문가들이 모인 예술과 기술 융합기반의 창작 그룹이다그동...
32699,1614,金지사 지역 출신 의원 초청 간담회SOC 지원 요청김영록 전남지사는 4일 서울 여의...,金지사 지역 출신 의원 초청 간담회SOC 지원 요청김영록 전남지사는 4일 서울 여의...
32700,1614,金지사 지역 출신 의원 초청 간담회SOC 지원 요청김영록 전남지사는 4일 서울 여의...,여의도에서 국회의원 초청 간담회를 열어 정책 현안과 2020년 정부예산안에 미반영...


In [184]:
split_dart = split_dart_df.rename_axis('id').reset_index()

In [187]:
id_to_corp = {}
for idx in range(len(split_dart)):
    id_to_corp[split_dart.iloc[idx]['id']] = split_dart.iloc[idx]['corp']

In [119]:
from transformers import (
    AutoTokenizer,
    BertModel, BertPreTrainedModel
)
import transformers

In [120]:
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")

In [182]:
train_df

,corp,plain,masked
0,354,구본영 시장 당선무효형 확정공약현안사업 이행 차질 우려 수장 공백 기강 해이 가능성...,구본영 시장 당선무효형 확정공약현안사업 이행 차질 우려 수장 공백 기강 해이 가능성...
1,354,구본영 시장 당선무효형 확정공약현안사업 이행 차질 우려 수장 공백 기강 해이 가능성...,무원들은 설마 했는데 올 것이 왔다는 분위기다일부 간부급 직원들은 예상했던 일이라면...
2,354,구본영 시장 당선무효형 확정공약현안사업 이행 차질 우려 수장 공백 기강 해이 가능성...,설 사업 관련 12개 시군 협의체 회장을 구 시장이 맡아왔다는 점에서 추진 동력을 ...
3,1302,오는 78일 후보 등록 15일 선거 실시 최종 선출자천 타천 23명 거론 중 이사와...,오는 78일 후보 등록 15일 선거 실시 최종 선출자천 타천 23명 거론 중 이사와...
4,1302,오는 78일 후보 등록 15일 선거 실시 최종 선출자천 타천 23명 거론 중 이사와...,거일 공고문을 지난달 30일 [MASK]문화원 홈페이지 및 게시판에 게시했다[MAS...
...,...,...,...
185434,9,신창균 재화성시문화재단은 2019 찾아가는 공연장을 위해 시민들로부터 신청을 받는다...,신창균 재화성시문화재단은 2019 찾아가는 공연장을 위해 시민들로부터 신청을 받는다...
185435,9,신창균 재화성시문화재단은 2019 찾아가는 공연장을 위해 시민들로부터 신청을 받는다...,을 신청할 수 있다고 밝혔다재단은 공연 운영이 가능한 실내 공간을 확보한 시민에 한...
185436,468,서울경제 아이템 스페셜 티저 포스터 2종이 전격 공개됐다주연 배우가 아닌 의미를 알...,서울경제 아이템 스페셜 티저 포스터 2종이 전격 공개됐다주연 배우가 아닌 의미를 알...
185437,468,서울경제 아이템 스페셜 티저 포스터 2종이 전격 공개됐다주연 배우가 아닌 의미를 알...,아이템을 향한 간절함이 담긴 인물의 이야기가 펼쳐질 것임을 예고하고 있다먼저 아이...


In [210]:
dart_tok = tokenizer( list(split_dart_df['txt']) ,padding = "max_length", return_tensors = "pt", truncation = True) 
train_tok = tokenizer( list(train_df['masked']) ,padding = "max_length", return_tensors = "pt", truncation = True)
valid_tok = tokenizer( list(valid_df['masked']) ,padding = "max_length", return_tensors = "pt", truncation = True)
test_tok = tokenizer(list(test_df['masked']),padding = "max_length", return_tensors = "pt", truncation = True)

In [212]:
train_corp_code = torch.tensor(train_df['corp'])
valid_corp_code = torch.tensor(valid_df['corp'])
test_corp_code = torch.tensor(test_df['corp'])

In [214]:
dart_tok['input_ids'].shape, train_tok['input_ids'].shape, train_corp_code.shape

(torch.Size([20643, 512]), torch.Size([152704, 512]), torch.Size([152704]))

In [215]:
dart_tok.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [216]:
from torch.utils.data import DataLoader, TensorDataset

In [218]:
train_set = TensorDataset(
    train_tok['input_ids'], train_tok['token_type_ids'], train_tok['attention_mask'], \
    train_corp_code)

valid_set = TensorDataset(
    valid_tok['input_ids'], valid_tok['token_type_ids'], valid_tok['attention_mask'], \
    valid_corp_code)

In [219]:
BATCH_SIZE = 16
dataloader = DataLoader(train_set, batch_size=BATCH_SIZE)

val_dataloader = DataLoader(valid_set, batch_size=BATCH_SIZE)


In [220]:
class BertEncoder(BertPreTrainedModel):

    def __init__(self, config):
        super(BertEncoder, self).__init__(config)
        self.bert = BertModel(config)
        self.init_weights()
    
    def forward(self,
            input_ids, 
            attention_mask=None,
            token_type_ids=None
        ): 
  
        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )
        
        pooled_output = outputs[1]
        return pooled_output

In [221]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [222]:
news_encoder = BertEncoder.from_pretrained("klue/bert-base").to(device)
dart_encoder = BertEncoder.from_pretrained("klue/bert-base").to(device)

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertEncoder: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at klue/bert-base were not used when initializing BertEncoder: ['cls.predictions.bi

In [223]:
from torch import optim
news_optimizer = optim.SGD(news_encoder.parameters(), lr=0.01, momentum=0.9)
dart_optimizer = optim.SGD(dart_encoder.parameters(), lr=0.01, momentum=0.9)

from torch.nn import CrossEntropyLoss

In [ ]:
from collections import Counter

epochs = 3
for e in range( epochs ):
    e_loss = 0
    for batch in tq.tqdm( dataloader ):
        news_data = {
            'input_ids' : batch[0].to(device).to(torch.int64), # ( B, seq_len )
            'token_type_ids' : batch[1].to(device).to(torch.int64),
            'attention_mask' : batch[2].to(device).to(torch.int64),
        }
        
        dart_data = {
            'input_ids' : batch[3].to(device).to(torch.int64), # ( B, seq_len )
            'token_type_ids' : batch[4].to(device).to(torch.int64),
            'attention_mask' : batch[5].to(device).to(torch.int64),
        }

        corp_table = batch[6]


        corp_code = corp_table.tolist()
        dic = {}


        batch_size = batch[0].shape[0]
        del batch
        
        news_encoder.train()
        news_encoder.zero_grad()
        news_emb = news_encoder(**news_data) # (B, hidden_dim)
        dart_encoder.train()
        dart_encoder.zero_grad()
        dart_emb = dart_encoder(**dart_data) # (B, hidden_dim)
        sim_score = torch.matmul( news_emb, dart_emb.T ) # (B , B)
        
        target = torch.arange(start=0, end = batch_size).to(device).to(torch.int64)
    
        CE_loss = CrossEntropyLoss()
        loss = CE_loss(sim_score, target)
        e_loss += loss.item()

        loss.backward()
        dart_optimizer.step()
        news_optimizer.step()

        del dart_emb
        del news_emb
        torch.cuda.empty_cache()
    print(e_loss / len(dataloader))